In [1]:
%matplotlib qt
%autosave 10
    
import matplotlib.pyplot as plt

import os
import numpy as np
import pandas as pd
import tifffile
from roifile import ImagejRoi
from PIL import Image
from pathlib import Path
from skimage.draw import polygon
from skimage.transform import resize
from skimage.measure import block_reduce
import cv2

import torch
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms

import pytorch_lightning as pl
from pytorch_lightning import Trainer, LightningModule


Autosaving every 10 seconds


In [2]:
# Path to the large TIFF image
base_path = Path('/home/cedric/coral')
image_path = base_path / 'T1_22.12.06.tif'

# Load the image using tifffile
full_image = tifffile.imread(image_path)
full_image = cv2.rotate(full_image, cv2.ROTATE_90_COUNTERCLOCKWISE)

print(f"Image shape: {full_image.shape}")

h, w, _ = full_image.shape


Image shape: (19326, 19155, 4)


In [3]:
def create_mask_from_roi(roi_files, image_shape):
    # Initialize mask as a 2D array
    mask = np.zeros(image_shape[:2], dtype=np.uint8)
    height, width = image_shape[:2]

    for roi_file in roi_files:
        roi_path = os.path.join(roi_dir, roi_file)
        roi = ImagejRoi.fromfile(roi_path)
        
        # Get the coordinates of the ROI
        coords = roi.coordinates()
        if coords is None or len(coords) == 0:
            continue  # Skip if no coordinates
        
        # Convert coordinates to NumPy array
        coords = np.array(coords, dtype=np.intp)
        x_coords = coords[:, 0]
        y_coords = coords[:, 1]
        
        # Ensure coordinates are within image bounds
        if x_coords.max() >= width or y_coords.max() >= height:
            print(f"Coordinates exceed image dimensions in {roi_file}")
            continue  # Skip or adjust the coordinates
        
        # Fill the polygon in the mask
        rr, cc = polygon(y_coords, x_coords, shape=mask.shape)
        mask[rr, cc] = 1  # Assign a label value (e.g., 1)
    
    return mask


In [4]:
# Directory containing ROI files
roi_dir = base_path / 'roi/'

# List of ROI files
roi_files = os.listdir(roi_dir)

# Create the mask
full_mask = create_mask_from_roi(roi_files, full_image.shape)
print("Mask created successfully!")


Mask created successfully!


In [5]:
if False:
    # Downsampling factor
    factor = 20
    
    # For a NumPy image array 'image' and downsampling factor 'factor'
    def downsample_image_nearest(image, factor):
        return image[::factor, ::factor, ...]
    
    # Example usage:
    image = downsample_image_nearest(full_image, factor=20)
    
    # Downsample the mask using block_reduce to preserve label values
    mask = block_reduce(full_mask, block_size=(factor, factor), func=np.max)

In [6]:
start_row = int(3/4*h)
end_row = -1
start_col = 0
end_col = int(1/4*w)

image = full_image[start_row:end_row, start_col:end_col]
mask = full_mask[start_row:end_row, start_col:end_col]

In [7]:
# Assuming 'downsampled_image' and 'downsampled_mask' are already defined
# 'downsampled_image' shape: (height, width, channels)
# 'downsampled_mask' shape: (height, width), values 0 or 1

# Define the overlay color (blue in this case)
overlay_color = np.array([0, 0, 255], dtype=np.float32)  # Blue color in float32
print("Overlay color shape:", overlay_color.shape)  # Should be (3,)

# Define the alpha blending factor
alpha = 0.5  # Adjust the transparency (0.0 to 1.0)

# Convert the image to float32 for blending and ensure it has 3 channels
image_float = image.astype(np.float32)[..., :3]
print("Image shape:", image_float.shape)  # Should be (height, width, 3)

# Create an overlay image filled with the overlay color
overlay = np.ones_like(image_float) * overlay_color  # Shape: (height, width, 3)

# Create a mask for blending and expand dimensions to match image channels
mask_3d = mask[:, :, np.newaxis]  # Shape: (height, width, 1)
print("Mask shape:", mask_3d.shape)  # Should be (height, width, 1)

# Perform alpha blending only on the masked areas
image_blend = np.where(mask_3d, 
                       (1 - alpha) * image_float + alpha * overlay, 
                       image_float)

# Convert back to uint8
image_with_overlay = image_blend.astype(np.uint8)

# Display the result
plt.figure(figsize=(12, 12))
plt.imshow(image_with_overlay)
plt.title('Downsampled Image with Masked Areas in Blue')
plt.axis('off')
plt.show()


Overlay color shape: (3,)
Image shape: (4831, 4788, 3)
Mask shape: (4831, 4788, 1)


In [9]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

# Assuming 'image' and 'mask' are already defined
# 'image' shape: (height, width, channels)
# 'mask' shape: (height, width), values 0 or 1

# Convert the mask to uint8 type with values 0 and 255
mask_uint8 = (mask * 255).astype(np.uint8)

# Find contours in the mask
contours, hierarchy = cv2.findContours(mask_uint8, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Create a copy of the image to draw contours on
image_with_contours = image.copy()

# Define the contour color (e.g., blue) and thickness
contour_color = (0, 0, 255)  # BGR format for OpenCV (blue color)
thickness = 2  # Thickness of the contour lines

# Draw contours on the image
cv2.drawContours(image_with_contours, contours, -1, contour_color, thickness)

# Display the result
plt.figure(figsize=(12, 12))
plt.imshow(image_with_contours)
plt.title('Image with Mask Outlines')
plt.axis('off')
plt.show()
